In [28]:
import json
import pandas as pd
from pandas import json_normalize

# Load the JSON data
with open('C:/Users/aober/Documents/Data Science Studium/3Semester/SoftwareProject3/osm-output/osm-output.json', 'r') as file:
    data = json.load(file)
    

with open('C:/Users/aober/Documents/Data Science Studium/3Semester/SoftwareProject3/PM3_1/data/colleges.json', 'r') as file:
    colleges_data = json.load(file)


In [29]:
nodes_data = data['nodes']

# Convert to DataFrame
df = pd.DataFrame(nodes_data)

colleges_df = pd.DataFrame(colleges_data)


In [31]:
# SHow all amenities
unique_amenities = df['amenity'].unique()

In [32]:
# SHow all addr:city
unique_amenities = df['addr:city'].unique()

In [33]:
# show all uni's / college's / research facilities
uni_df = df[df['amenity'].isin(['university', 'college', 'research_facility'])]
#print(uni_df.name.unique())

In [34]:
uni_df = colleges_df

In [35]:
# Keeping only amentities which are relevant to us
amenities_list = [
    'bar', 'cafe', 'fast_food', 'food_court', 'restaurant', 'pub', 
    'arts_centre', 'casino', 'cinema', 'events_venue', 'music_venue', 
    'nightclub', 'theatre', 'library', 'atm', 'bank', 
    'bus_station', 'bicycle_parking', 'bicycle_repair_station', 'doctors', 'hospital', 'pharmacy', 'police', 'post_box', 
    'post_office'
]

# Filter the dataframe based on the amenities list
filtered_df = df[df['amenity'].isin(amenities_list)]


In [36]:
# This function will be used for the process beneath, to filter the data by distance of all university facilities
import math

def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radius of the Earth in kilometers
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) ** 2 +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c


In [37]:
filtered_df.loc[:, 'lat'] = filtered_df['lat'].astype(float)
filtered_df.loc[:, 'lon'] = filtered_df['lon'].astype(float)

In [38]:
#This will filter the data
filtered_df_copy = filtered_df.copy()

# Ensure 'lat' and 'lon' are float types
filtered_df_copy['lat'] = filtered_df_copy['lat'].astype(float)
filtered_df_copy['lon'] = filtered_df_copy['lon'].astype(float)

# Extract coordinates for universities, colleges, or research facilities
facilities = uni_df
facilities_coords = facilities[['lat', 'lon']].values

# For each amenity, compute the distance to the nearest facility
def min_distance_to_facility(row):
    return min(haversine(row['lat'], row['lon'], facility[0], facility[1]) for facility in facilities_coords)

# Use the .loc[] accessor on the copy
filtered_df_copy['distance_to_facility'] = filtered_df_copy.apply(min_distance_to_facility, axis=1)

# Keep only amenities which are university, college, research_institute or closer than 5km to them
final_df = filtered_df_copy[(filtered_df_copy['amenity'].isin(['university', 'college', 'research_institute'])) | (filtered_df_copy['distance_to_facility'] <= 10)]



In [50]:
final_df.loc[:, 'location'] = final_df.apply(lambda row: {"type": "Point", "coordinates": [row['lon'], row['lat']]}, axis=1)


In [56]:
print(final_df.apply(lambda row: {"type": "Point", "coordinates": [row['lon'], row['lat']]}, axis=1))

14        {'type': 'Point', 'coordinates': [7.5054728, 4...
17        {'type': 'Point', 'coordinates': [8.9422989, 4...
18        {'type': 'Point', 'coordinates': [8.9424857, 4...
19        {'type': 'Point', 'coordinates': [9.3752645, 4...
46        {'type': 'Point', 'coordinates': [8.5191262, 4...
                                ...                        
233701    {'type': 'Point', 'coordinates': [7.3542359, 4...
233760    {'type': 'Point', 'coordinates': [8.8014349, 4...
233761    {'type': 'Point', 'coordinates': [8.8015298, 4...
233823    {'type': 'Point', 'coordinates': [6.5904091, 4...
233827    {'type': 'Point', 'coordinates': [6.6444685, 4...
Length: 34934, dtype: object


In [59]:
import numpy as np

# Convert the DataFrame back to a dictionary
data_dict = final_df.to_dict(orient='records')

# Clean the dictionary to remove NaN values
cleaned_data = []
for record in data_dict:
    clean_record = {key: value for key, value in record.items() if not (isinstance(value, float) and np.isnan(value))}
    cleaned_data.append(clean_record)
# To create the same nde structure as before



In [60]:
#structured_json.to_json('C:/Users/aober/Documents/Data Science Studium/3Semester/SoftwareProject3/PM3_1/data/processed_data.json', orient='records', lines=True)
output_path = 'C:/Users/aober/Documents/Data Science Studium/3Semester/SoftwareProject3/PM3_1/data/processed_data.json'
with open(output_path, 'w') as file:
    json.dump(cleaned_data, file, indent=4)
